In this file, patient conditions (diseases) are queried to find all patients with a history of cancer who also have cardiac issues (not assessed chronoclogically yet). I also compiled a list of chemotherapies from the medications.csv file and looked to see how many patients have been exposed to chemo.

In [2]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [3]:
os.chdir('C:/Users/Student/Documents/EHR---Team-6-Project')

In [4]:
# Load the conditions.csv df into a list and determine unique values
conditions = pd.read_csv('conditions.csv')

r_time = pd.read_csv('r_time.csv')

chemo_patients = pd.read_csv('chemo_patients.csv')

cardiac_patients = pd.read_csv('cardiac_patients.csv')

cancer_patients = pd.read_csv('cancer_patients.csv')

patients = pd.read_csv('PatientData.csv')

In [5]:
condition_list = ['Body mass index 30+ - obesity (finding)', 'Prediabetes',
       'Anemia (disorder)', 'Hypertension', 'Diabetes',
       'Hypertriglyceridemia (disorder)',
       'Metabolic syndrome X (disorder)', 'Hyperglycemia (disorder)',
       'Chronic obstructive bronchitis (disorder)', 'Hyperlipidemia',
       'Chronic kidney disease stage 1 (disorder)',
       'Stroke',
       'Shock (disorder)',
       'Neuropathy due to type 2 diabetes mellitus (disorder)',
       'Diabetic retinopathy associated with type II diabetes mellitus (disorder)',
       'Pulmonary emphysema (disorder)',
       'Alcoholism', 
       'Nonproliferative diabetic retinopathy due to type 2 diabetes mellitus (disorder)',
       'Macular edema and retinopathy due to type 2 diabetes mellitus (disorder)',
       'Proliferative diabetic retinopathy due to type II diabetes mellitus (disorder)',
       'Rheumatoid arthritis',
       'Body mass index 40+ - severely obese (finding)',
       'Opioid abuse (disorder)', 
       'Primary fibromyalgia syndrome', 
       'Chronic kidney disease stage 2 (disorder)',
       'Microalbuminuria due to type 2 diabetes mellitus (disorder)',
       'Blindness due to type 2 diabetes mellitus (disorder)',
       'Cystic Fibrosis',
       'Diabetes from Cystic Fibrosis',
       'Acute respiratory failure (disorder)', 'Lupus erythematosus',
       'Chronic kidney disease stage 3 (disorder)',
       'Proteinuria due to type 2 diabetes mellitus (disorder)']

In [6]:
#Creating dataframes having equivalent number of rows in order to join together, will filter to appropriate date ranges in later step
cancer_patients_repeated = pd.concat([cancer_patients] * len(r_time), ignore_index=True)
r_time_repeated = pd.concat([r_time] * len(cancer_patients), ignore_index=True)

In [7]:
#Dropping duplicates
chemo_patients = chemo_patients.drop_duplicates()

In [8]:
#Effectively, joining our repeated member month table to our repeated cancer patients table 
cancer_patients_repeated['PatMonth'] = r_time_repeated['PatMonth']

In [9]:
#Converting the START column to datetime
cancer_patients_repeated['START'] = pd.to_datetime(cancer_patients_repeated['START'])

In [10]:
#Creating First DOM Field for Filter
cancer_patients_repeated['START_FLOOR'] = (cancer_patients_repeated['START'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))

In [11]:
#Filtering down to rows only between start and stop date values
PatMonths = cancer_patients_repeated.query('PatMonth >= START_FLOOR and PatMonth <= STOP').sort_values(by = ['PatMonth', 'PATIENT']).reset_index()

#Dropping unneeded columns
PatMonths.drop(columns=PatMonths.columns[[0, -1]], axis = 1, inplace = True)

In [12]:
#Transforming into one hot encoded values
PatMonthsWide = PatMonths[['PATIENT', 'START', 'STOP', 'PatMonth']].join(pd.get_dummies(PatMonths['DESCRIPTION'])).groupby(['PATIENT', 'START', 'STOP', 'PatMonth']).max().reset_index()

In [13]:
#Joining Patient Data
PatMonthsWide = PatMonthsWide.merge(patients, left_on = 'PATIENT', right_on = 'Id', how = 'inner')

In [14]:
#Joining Cardiac and MemMonths
PatMonthsWide = PatMonthsWide.merge(cardiac_patients, on = ['PATIENT', 'PatMonth'], how = 'left')

In [15]:
#Converting the BIRTHDATE column to datetime
PatMonthsWide['BIRTHDATE'] = pd.to_datetime(PatMonthsWide['BIRTHDATE'])

PatMonthsWide['PatMonth'] = pd.to_datetime(PatMonthsWide['PatMonth'])

PatMonthsWide['DEATHDATE'] = pd.to_datetime(PatMonthsWide['DEATHDATE'])

In [16]:
#Creating Age Field
PatMonthsWide['Age'] = PatMonthsWide.apply(lambda x: relativedelta(x['PatMonth'], x['BIRTHDATE']).years, axis=1)

In [17]:
#Creating Deceased Flag
PatMonthsWide['DeceasedFlag'] = (PatMonthsWide['PatMonth'] == (PatMonthsWide['DEATHDATE'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))).astype(int)

In [18]:
#Dropping rows after patient had passed
PatMonthsWide = PatMonthsWide.query('PatMonth <= DEATHDATE').reset_index()

In [19]:
#Dropping Unneeded Fields
PatMonthsWide.drop(['Id', 'BIRTHDATE', 'DEATHDATE', 'START', 'index'], axis = 1, inplace = True)

In [20]:
#Converting to appropriate data type for join
chemo_patients['PatMonth'] = pd.to_datetime(chemo_patients['PatMonth'])

In [21]:
#Joining Chemo Patients to Pat Month Table
PatMonthsWide = PatMonthsWide.merge(chemo_patients, on = ['PATIENT', 'PatMonth'], how = 'left')

In [22]:
#Filling NaN values with 0s
PatMonthsWide.fillna(0, inplace = True)

In [23]:
#Reducing conditions dataframe to only include patient data with chemo & storing in Condition_df
Condition_df = conditions[conditions['PATIENT'].isin(chemo_patients['PATIENT'])==True].reset_index()

In [24]:
#Changing NaN values to Current Date
Condition_df['STOP'] = Condition_df['STOP'].replace(np.nan, '2021-08-01')

In [25]:
#Converting the STOP column to datetime
Condition_df['STOP'] = pd.to_datetime(Condition_df['STOP'])

In [26]:
#Converting the START column to datetime
Condition_df['START'] = pd.to_datetime(Condition_df['START'])

In [27]:
#Creating dataframes having equivalent number of rows in order to join together
#This is filtered to appropriate date ranges in later step
Condition_df_repeated = pd.concat([Condition_df] * len(r_time), ignore_index=True)
r_time_repeated_Cond = pd.concat([r_time] * len(Condition_df), ignore_index=True)

In [28]:
#Effectively, joining our repeated patient month table to our repeated cancer patients table 
Condition_df_repeated['PatMonth'] = r_time_repeated_Cond['PatMonth']

In [29]:
#Creating First DOM Field for Filter
Condition_df_repeated['START_FLOOR'] = (Condition_df_repeated['START'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))

In [30]:
#Filtering down to rows only between start and stop date values
PatMonthsCond = Condition_df_repeated.query('PatMonth >= START_FLOOR and PatMonth <= STOP').sort_values(by = ['PatMonth', 'PATIENT']).reset_index()

In [31]:
#Dropping unneeded columns
PatMonthsCond.drop(columns=PatMonthsCond.columns[[0, 1, -1]], axis = 1, inplace = True)

In [32]:
#Transforming into one hot encoded values
PatMonthsWideCond = PatMonthsCond[['PATIENT', 'START', 'STOP', 'PatMonth']].join(pd.get_dummies(PatMonthsCond['DESCRIPTION'])).groupby(['PATIENT', 'START', 'STOP', 'PatMonth']).max().reset_index()

In [33]:
#Filling NaN values with 0s
PatMonthsWideCond.fillna(0, inplace = True)

In [34]:
#Dropping START and STOP Fields
PatMonthsWideCond.drop(['START', 'STOP'], axis = 1, inplace = True)

In [35]:
#Collapsing Rows
PatMonthsWideCond = PatMonthsWideCond.groupby(['PATIENT', 'PatMonth']).sum().reset_index()

In [36]:
#Dropping fields that would otherwise be duplicated following join
PatMonthsWideCond.drop(['Carcinoma in situ of prostate (disorder)', 'Malignant neoplasm of breast (disorder)', 'Malignant tumor of colon', 'Metastasis from malignant tumor of prostate (disorder)', 'Neoplasm of prostate', 'Non-small cell carcinoma of lung  TNM stage 1 (disorder)', 'Non-small cell lung cancer (disorder)', 'Primary malignant neoplasm of colon', 'Primary small cell malignant neoplasm of lung  TNM stage 1 (disorder)', 'Secondary malignant neoplasm of colon', 'Suspected lung cancer (situation)'], axis = 1, inplace = True)

In [37]:
#Dropping fields that would otherwise be duplicated following join
PatMonthsWideCond.drop(['Chronic congestive heart failure (disorder)', 'Cardiac Arrest', 'History of cardiac arrest (situation)', 'Atrial Fibrillation', 'Coronary Heart Disease', 'Myocardial Infarction', 'History of myocardial infarction (situation)'], axis = 1, inplace = True)

In [38]:
#Converting to appropriate data type for join
PatMonthsWideCond['PatMonth'] = pd.to_datetime(PatMonthsWideCond['PatMonth'])

In [39]:
#Combining Conditions with Core Patient Month Dataset
NewPatMonths = PatMonthsWide.merge(PatMonthsWideCond, on = ['PATIENT','PatMonth'], how = 'left')

In [40]:
#Engineering new field to represent occurance of adverse caridac event
NewPatMonths['Cardiac Event'] = NewPatMonths[['Chronic congestive heart failure (disorder)', 'Cardiac Arrest', 'History of cardiac arrest (situation)', 'Atrial Fibrillation', 'Coronary Heart Disease', 'Myocardial Infarction', 'History of myocardial infarction (situation)']].any(axis = 1).astype(int)

In [41]:
#Creating Patient Year field
NewPatMonths['PatYear'] = NewPatMonths['PatMonth'].dt.strftime('%Y')

In [42]:
#Dropping fields no longer necessary; determined that dataset would be collapsed at the patient year level
NewPatMonths.drop(['PatMonth', 'STOP'], axis = 1, inplace = True)

In [55]:
#Grouping by patient and year, taking the max one hot encoded value for that period
NewPatYear = NewPatMonths.groupby(['PATIENT', 'PatYear']).max()

In [57]:
#Resetting Index
NewPatYear.reset_index(inplace = True)

In [58]:
#Converting float dataypes to align with integers in dataframe
NewPatYear = NewPatYear.convert_dtypes(convert_floating = True)

In [60]:
#Exporting to csv
NewPatYear.to_csv(r'NewPatYearTest.csv') 